### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = hoje

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-11-11



100%|██████████| 249/249 [03:51<00:00,  1.08it/s]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 219 jogos.

----------------------------------------------------------

Atenção! 30 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
219,2023-11-11,Argentina Liga A,19:00,Platense,Zarate,66.0,56.0,4º Quarto4'
220,2023-11-11,Eua Ncaa,19:00,Seattle,Montana State,NaN,NaN,Não iniciado
221,2023-11-11,Eua Ncaa,19:00,Mississippi St.,Tenn. Martin,NaN,NaN,Não iniciado
222,2023-11-11,Eua Ncaa,19:30,Winthrop,Drexel,NaN,NaN,Não iniciado
223,2023-11-11,Chile Copa Do Chile,19:45,Liceo Pablo Neruda,Leones Quilpue,34.0,84.0,4º Quarto1'
224,2023-11-11,Argentina Liga A,20:00,Instituto de Cordoba,Independiente de Oliva,37.0,37.0,Intervalo
225,2023-11-11,Eua Ncaa,20:00,Providence,Wisc. Milwaukee,NaN,NaN,Não iniciado
226,2023-11-11,Eua Nba,20:00,Orlando Magic,Milwaukee Bucks,47.0,38.0,2º Quarto6'
227,2023-11-11,Chile Copa Do Chile,20:00,Puerto Montt,CD Puerto Varas,38.0,41.0,Intervalo
228,2023-11-11,Eua Ncaa,20:00,Cleveland State,Ohio,NaN,NaN,Não iniciado
